In [ ]:
# Setup scenario
import copy

from constants import DATA_ENTITIES_DIR, DATA_HAZARDS_DIR, DATA_TEMP_DIR, REQUIREMENTS_DIR
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from climada.engine import CostBenefit, Impact, ImpactCalc
from climada.engine.cost_benefit import risk_aai_agg
from climada.entity import DiscRates, Entity, Exposures, LitPop
from climada.entity.impact_funcs import ImpactFunc, ImpactFuncSet, ImpfTropCyclone
from climada.entity.impact_funcs.storm_europe import ImpfStormEurope
from climada.entity.impact_funcs.trop_cyclone import ImpfSetTropCyclone
from climada.entity.measures import Measure, MeasureSet
from climada.hazard import Hazard
from climada.util.api_client import Client

from costben.costben_handler import CostBenefitHandler
from exposure.exposure_handler import ExposureHandler
from hazard.hazard_handler import HazardHandler
from impact.impact_handler import ImpactHandler

import handlers
from logger_config import LoggerConfig

logger = LoggerConfig(logger_types=["file"])

costben_handler = CostBenefitHandler()
exposure_handler = ExposureHandler()
hazard_handler = HazardHandler()
impact_handler = ImpactHandler()

client = Client()


In [ ]:
haz_present = client.get_hazard(
    "earthquake",
    properties = {
        "country_name": "Egypt",
    }
)

# haz_future = client.get_hazard(
#     "earthquake",
#     properties = {
#         "country_name": "Egypt",
#         "climate_scenario": "rcp60",
#         "year_range": "2030_2050"
#     }
# )

In [ ]:
future_year = 2040

entity_present = exposure_handler.get_entity_from_xlsx("entity_template_simple_egy.xlsx")
# impf_set = entity_present.impact_funcs
# measure_set = entity_present.measures
# disc_rates = entity_present.disc_rates
# exp_present = entity_present.exposures
# exp_present.ref_year
# exp_future = copy.deepcopy(exp_present)
# exp_future.ref_year = future_year
# n_years = exp_future.ref_year - exp_present.ref_year + 1
# growth_rate = 1.02
# growth = growth_rate ** n_years
# exp_future.gdf['value'] = exp_future.gdf['value'] * growth
# entity_future = Entity(exposures=exp_future, disc_rates=disc_rates,
#                        impact_func_set=impf_set, measure_set=measure_set)

# year_range = np.arange(exp_present.ref_year, exp_future.ref_year + 1)
# annual_discount_zero = np.zeros(n_years)
# annual_discount_stern = np.ones(n_years) * 0.014

# discount_zero = DiscRates(year_range, annual_discount_zero)
# discount_stern = DiscRates(year_range, annual_discount_stern)

In [ ]:
entity_present.impact_funcs.get_func('EQ')[0].calc_mdr(0)

In [ ]:
from climada.engine import CostBenefit
from climada.engine.cost_benefit import risk_aai_agg

costben_measures_only = CostBenefit()
costben_measures_only.calc(haz_present, entity_present, haz_future=None, ent_future=None,
                           future_year=future_year, risk_func=risk_aai_agg, imp_time_depen=None, save_imp=True)

In [ ]:
combined_costben = costben_measures_only.combine_measures(['Measure A', 'Measure B'],
                                                          'Combined measures',
                                                          new_color=np.array([0.1, 0.8, 0.8]),
                                                          disc_rates=discount_zero)

In [ ]:
ax = costben_measures_only.plot_event_view((25, 50, 100, 250))

In [ ]:
costben = CostBenefit()
costben.calc(haz_present, entity_present, haz_future=haz_future, ent_future=entity_future,
             future_year=future_year, risk_func=risk_aai_agg, imp_time_depen=1, save_imp=True)

In [ ]:
# define this as a function because we'll use it again later
def waterfall():
  return costben.plot_waterfall(haz_present, entity_present, haz_future, entity_future,
                                risk_func=risk_aai_agg)

ax = waterfall()